![Slide 1](./images/Slide2.JPG)


![Slide 2](./images/Slide3.JPG)


![Slide 3](./images/Slide4.JPG)


![Slide 4](./images/Slide5.JPG)


![Slide 5](./images/Slide6.JPG)


![Slide 6](./images/Slide7.JPG)


In [3]:
#!/usr/bin/env python3

def ask_multiline_question(query: str) -> str:
    """
    Ask a question that can span multiple lines.
    The user enters lines until a blank line is entered.
    """
    print(query)
    lines = []
    while True:
        line = input()
        if not line.strip():  # Stop reading on an empty line
            break
        lines.append(line)
    return "\n".join(lines).strip()


def ask_question(query: str) -> str:
    """Ask a single-line question and return the answer."""
    return input(query)


def run():
    # 1. SET UP REPORT TOPIC
    # Get initial multi-line query from the user
    initial_query = ask_multiline_question(
        "What would you like to research? (Enter blank line to finish)"
    )

    # Get breadth parameter (with default of 4 if input is empty or invalid)
    breadth_input = ask_question("Enter research breadth (recommended 2-10, default 4): ")
    try:
        breadth = int(breadth_input)
    except ValueError:
        breadth = 4

    # Get depth parameter (with default of 2 if input is empty or invalid)
    depth_input = ask_question("Enter research depth (recommended 1-5, default 2): ")
    try:
        depth = int(depth_input)
    except ValueError:
        depth = 2

    print("Creating research plan...")
    print(initial_query)
    print(f"breadth={breadth}, depth={depth}")


In [4]:
run()

What would you like to research? (Enter blank line to finish)


 howto becomemillionaire and make lots of money in a week
 
Enter research breadth (recommended 2-10, default 4):  2
Enter research depth (recommended 1-5, default 2):  2


Creating research plan...
howto becomemillionaire and make lots of money in a week
breadth=2, depth=2


In [11]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import Field, create_model
from typing import List
import os
from datetime import datetime
from dotenv import load_dotenv

load_dotenv(dotenv_path='./.env-local')

def system_prompt_1():
    """
    Generates a system prompt for API requests, including the current timestamp.
    """
    now = datetime.utcnow().isoformat()
    return (
        f"You are an expert researcher. Today is {now}. Follow these instructions when responding:\n"
        f"- You may be asked to research subjects that is after your knowledge cutoff, assume the user is right when presented with news.\n"
        f"- The user is a highly experienced analyst, no need to simplify it, be as detailed as possible and make sure your response is correct.\n"
        f"- Be highly organized.\n"
        f"- Suggest solutions that I didn't think about.\n"
        f"- Be proactive and anticipate my needs.\n"
        f"- Treat me as an expert in all subject matter.\n"
        f"- Mistakes erode my trust, so be accurate and thorough.\n"
        f"- Provide detailed explanations, I'm comfortable with lots of detail.\n"
        f"- Value good arguments over authorities, the source is irrelevant.\n"
        f"- Consider new technologies and contrarian ideas, not just the conventional wisdom.\n"
        f"- You may use high levels of speculation or prediction, just flag it for me."
    )

# Function to dynamically create JSON Schema Definition using Pydantic
def create_followup_model(max_questions: int):
    return create_model(
        'FollowUpQuestions',
        questions=(List[str], Field(description="List of follow-up questions.", max_items=max_questions))
    )

# Setup the Langchain Chat Model explicitly using GPT-4 JSON mode
llm = ChatOpenAI(
    model=os.getenv("OPENAI_MODEL", "o3-mini"),
    base_url=os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1"),
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.5,
    model_kwargs={"response_format": {"type": "json_object", "reasoning_effort": "medium"}}
)

# Prompt Templates
user_template = "Given the following query from the user, ask some follow-up questions to clarify the research direction. Return a maximum of {num_questions} questions. User query: {query}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_1()),
    ("human", user_template)
])

# Async function to generate follow-up questions
async def generate_feedback(query: str, num_questions: int = 3) -> List[str]:
    FollowUpQuestions = create_followup_model(num_questions)
    chain = prompt | llm.with_structured_output(FollowUpQuestions)
    response = await chain.ainvoke({"query": query, "num_questions": num_questions})
    return response.questions[:num_questions]

# Async helper function for multiline user input
async def ask_multi_line_question(question: str) -> str:
    print(question)
    print("(Submit an empty line to finish your answer.)")
    lines = []
    while True:
        line = input()
        if not line.strip():
            break
        lines.append(line)
    return "\n".join(lines).strip()

# Main script execution
async def run():
    initial_query = "can pigs fly"
    breadth = 3
    depth = 2

    # Generate follow-up questions
    follow_up_questions = await generate_feedback(initial_query, breadth)

    # Collect user answers
    answers = []
    for question in follow_up_questions:
        answer = await ask_multi_line_question(f"\n{question}\nYour answer:")
        answers.append(answer)

    # Combine query with user answers
    combined_query = f"Initial query: {initial_query}\n"
    combined_query += "\n".join(f"Q: {q}\nA: {a}" for q, a in zip(follow_up_questions, answers))

    print("\nPrinting User Prompt with Q & A...")
    print(combined_query)
    print(f"breadth={breadth}, depth={depth}")

ModuleNotFoundError: No module named 'dotenv'

In [12]:
!pip install dotenv
